In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
from lsst.summit.utils.plotting import plot
from lsst.obs.lsst import LsstCam
import lsst.afw.cameraGeom.utils as camGeomUtils
import lsst.afw.math as afwMath
import lsst.afw.display as afwDisplay
from lsst.afw import image
from lsst.geom import Point2I
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
import cv2

In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation'])
camera = LsstCam.getCamera()
instrument = "LSSTCam"

# First run one CCD as a test

In [ ]:
dayObs = 20250909
seqNum = 313
expId = int(dayObs * 1E5 + seqNum)
detNum = 106
if detNum < 0:
    detName = 'R13_S01'
    det = camera[detName]
    detNum = det.getId()
print(detNum)
calexp = butler.get('preliminary_visit_image', detector=detNum, visit=expId, instrument=instrument)
#raw = butler.get('raw', detector=detNum, exposure=expId, instrument=instrument)

In [ ]:
type(calexp)

In [ ]:
%matplotlib inline
x = plot(calexp, stretch='ccs')
ax = x.get_axes()[0]
ax.set_title(f"{expId} {detNum}")
x.savefig(f"/home/c/cslage/u/Satellites/images/LSSTCam_{expId}_{detNum}.png")
x

# The cell below does the streak finding work.

In [ ]:
def find_faint_ridges(exp, bin=4, kernel=11, sigma=12.0, edge=20, 
                      threshold=-0.05, aspect=8, streak_width=100, 
                      hor_vert_limit=0.05, output="L"):
    """
    Parameters
    ----------
    exp : `lsst.afw.image._exposure.ExposureF`
        Input exposure to search for streaks
    bin : `int`
        Image is initially binned bin x bin.
    kernel : `int`
        Size in pixels of Gaussian kernel for initial smoothing.
    sigma: 'float'
        Sigma is the standard deviation of the Gaussian kernel 
        used to compute the Hessian second derivatives. 
        Which is used as a weighting function for the 
        auto-correlation matrix.
    edge: 'int'
        Binned image has edge pixels zero'd out before thresholding.
    threshold: 'float'
        Threshold applied to the Hessian minima_ridges eigenvalue
        array to identify possible streaks.
    aspect: 'float'
        After thresholding, only regions with aspect ratio
        greater than aspect are kept.  This eliminates roundish
        regions from stars.
    streak_width: 'int'
        If output = "I" or "P", the width os the streak which is
        drawn on the image
    hor_vert_limit: 'float'
        This eliminates streaks which are nearly horizontal or 
        nearly vertical.
    output: 'str'
        Determines wht the code retruns.  See below for details.

    Returns
    -------
    Several option for what is returned:
    output == "L"
        lines : `list`
            list of [Point2I, Point2I] that are endpoints
            of the streak,
    output == "I"
        disp_img : `numpy.ndarray`
        An array the same size as the original image,
        with the detected streaks added.  This is useful for 
        using showCamera to plot the whole focal plane.
    output == "P"
        fig : `matplotlib.figure.Figure`
        A plot of the original image and the 
        detected streaks.
    """    
    arr = exp.image.array    
    # Bin original image down to binxbin pixels
    arr = np.clip(arr, a_min=0, a_max=100)
    new_shape = (int(arr.shape[0] / bin), int(arr.shape[1] / bin))
    # Rebin by averaging
    bin_arr = arr.reshape(
        new_shape[0],
        arr.shape[0] // new_shape[0],
        new_shape[1],
        arr.shape[1] // new_shape[1]
    ).mean(-1).mean(1)
    
    # Use the Hessian matrix to find streaks
    # The minima ridges output has been most effective
    # in finding the streaks
    gauss = cv2.GaussianBlur(bin_arr, (kernel, kernel), 0) # Blur with Gaussian kernel
    H_elems = hessian_matrix(gauss, sigma=sigma, order='rc', use_gaussian_derivatives=False)
    maxima_ridges, minima_ridges = hessian_matrix_eigvals(H_elems)
    # Now we create a binary image 
    # Setting this threshold has been tricky
    binary_ridges = minima_ridges < threshold
    binary_ridges = binary_ridges.astype(np.uint8)
    # Set edges of binary_ridges to zero
    binary_ridges[:,0:edge] = 0
    binary_ridges[:,-edge:-1] = 0
    binary_ridges[0:edge,:] = 0
    binary_ridges[-edge:-1,:] = 0
    # Convert to 0 -> 255
    _, binary = cv2.threshold(binary_ridges, 0.5, 255, cv2.THRESH_BINARY)
    # Find connected regions
    num_labels, labels, stats, centroids = \
        cv2.connectedComponentsWithStats(binary, connectivity=8)
    # Sort to find regions with long aspect ratios
    long_labels = []
    for i in range(num_labels):
        mask = np.uint8(labels == i)
        # Extract points (x,y) of this component
        ys, xs = np.where(mask > 0)
        points = np.column_stack((xs, ys))
        rect = cv2.minAreaRect(points)
        (center, (width, height), angle) = rect
        if height > 0:
            aspect_ratio = max(width, height) / min(width, height)
        else:
            aspect_ratio = 0  # Handle division by zero for flat regions
        if aspect_ratio > aspect:
            long_labels.append(i)

    c, r = arr.shape
    # Fit lines to the longest ones
    lines = []
    for label in long_labels:
        mask = np.uint8(labels == label)
        # Extract points (x,y) of this component
        ys, xs = np.where(mask > 0)
        points = np.column_stack((xs, ys))
        # Fit a line through the points
        # x0, y0 are shape centroid
        # vx, vy are a normlized vector in the direction of the line
        # Resize x0 and y0 to the original image
        [vx, vy, x0, y0] = cv2.fitLine(points, cv2.DIST_L2, 0, 0.01, 0.01)
        vx = vx[0]; vy = vy[0]; x0 = x0[0] * bin; y0 = y0[0] * bin
        print(x0, y0, vx, vy)
        # Weed out near horizontal or vertical lines
        if (abs(vx) < hor_vert_limit) or (abs(vy) < hor_vert_limit):
            continue
        # Find points at the edges
        alpha = min((r - 1 - x0) / vx, (c - 1 - y0) / vy)
        right_point = (int(x0 + alpha * vx), int(y0 + alpha * vy))
        beta = min(x0 / vx , y0 / vy)
        left_point = (int(x0 - beta * vx), int(y0 - beta * vy))
        lines.append([Point2I(left_point), Point2I(right_point)])

    if output == "L":
        return lines

    if output in ["I", "P"]:
        disp_img = np.zeros_like(arr)
        for [right_point, left_point] in lines:
            cv2.line(disp_img, right_point, \
                     left_point, (255,255,255), streak_width)
    if output == "I":
        return disp_img

    if output == "P":
        fig, axes = plt.subplots(2, 2, figsize=(10, 8))
        plt.subplots_adjust(hspace=0.3)
        plt.suptitle(f"Streak finding {expId}_{detNum}")
        ax = axes.ravel()
        ax[0].set_title('Original Image')
        ax[0].imshow(arr, cmap=plt.cm.gray, vmin=0, vmax=100, origin='lower')
        ax[1].set_title(f'Minima Ridges (sigma={sigma})\n Negative Hessian eigenvalues')
        ax[1].imshow(minima_ridges, origin='lower', vmin=-.01, vmax=.01)
        ax[2].set_title(f'Binary Ridges (sigma={sigma})\n Thresholding minima_ridges')
        ax[2].imshow(binary_ridges, origin='lower', cmap=plt.cm.gray, vmin=0, vmax=1)
        ax[3].set_title(f'Detected Streaks (sigma={sigma})\nAfter finding longest regions')
        ax[3].imshow(disp_img, cmap=plt.cm.gray, origin='lower')
    return fig
        

In [ ]:
lines = find_faint_ridges(calexp, output="L")

In [ ]:
# Example usage:
%matplotlib inline
fig = find_faint_ridges(calexp, output="P")

In [ ]:
# This eliminates the heavily vignetted corners from the whole camera plot
rafts = [      'R01','R02','R03', 
         'R10','R11','R12','R13','R14',
         'R20','R21','R22','R23','R24',
         'R30','R31','R32','R33','R34',  
               'R41','R42','R43']
ccds = ['S00','S01','S02',
        'S10','S11','S12',
        'S20','S21','S22']
corners = ['R01_S00', 'R01_S01', 'R03_S01', 'R03_S02', \
                     'R10_S00', 'R10_S10', 'R30_S20', 'R30_S10', \
                    'R41_S20', 'R41_S21', 'R43_S21', 'R43_S22', \
                    'R34_S22', 'R34_S12', 'R14_S02', 'R14_S12']
detectorNameList = []
for raft in rafts:
    for ccd in ccds:
        name = raft+'_'+ccd
        if name not in corners:
            detectorNameList.append(name)

In [ ]:
def streakCallback(im, ccd, imageSource):
    # This runs the streak finding algorithm on each CCD
    calexp = butler.get('preliminary_visit_image', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    disp_img = find_faint_ridges(calexp, output="I")
    print(ccd.getId(), np.max(disp_img.flatten())) # This will flag which CCDs have streaks
    oim = image.ImageF(array=disp_img, deep=False, xy0=Point2I(0, 0))
    return oim

# This assembles the entire camera image

In [ ]:
%matplotlib inline
instrument = "LSSTCam"
camera = butler.get('camera', instrument=instrument)
fig = plt.figure(figsize=(12,12))
disp = afwDisplay.Display(1, "matplotlib")
disp.scale('linear', min=0, max=100)
disp.setImageColormap("gray")
dataType='raw'
mos = camGeomUtils.showCamera(camera,
                              camGeomUtils.ButlerImage(butler, dataType, 
                                                       instrument=instrument, 
                                                       day_obs=dayObs, seq_num=seqNum,
                                                       verbose=False, callback=streakCallback,
                                                       background=np.nan),
                              detectorNameList=detectorNameList,
                              binSize=16, display=disp, overlay=False,
                              title="%d %d" % (dayObs, seqNum))

plt.savefig(f"/home/c/cslage/u/Satellites/streak_images/Streak_Finding_{dayObs}_{seqNum}.png")

In [ ]:
x0, y0, vx, vy = 1401.5178, 1712.9017, 0.5488743, 0.8359049
theta = -np.atan2(vx, vy)
rho = -380.0
x1 = x0 + 500 * vx
y1 = y0 + 500 * vy
(x3, y3), (x4, y4) = (277, 0), (2906, 4004)
plt.imshow(calexp.image.array, cmap=plt.cm.gray, vmin=0, vmax=100, origin='lower')
plt.text(2048, 2000, "o", size=20, color='magenta', horizontalalignment='center', verticalalignment='center')
plt.text(x0-800, y0, "(x1, y1)", color='yellow', verticalalignment='center')
plt.text(x0, y0, "X", size=20, color='yellow', horizontalalignment='center', verticalalignment='center')
plt.text(x1, y1, "X", size=20, color='yellow', horizontalalignment='center', verticalalignment='center')
plt.text(x1-800, y1, "(x2, y2)", color='yellow', verticalalignment='center')
plt.plot([2048, 2048+rho*np.cos(theta)], [2000, 2000+rho*np.sin(theta)], lw=2, color='magenta')
plt.text(1900, 2200, f"rho = {rho:.1f}", color='magenta')
plt.plot([2048, 4096], [2000, 2000], lw=2, ls='--', color='magenta')
plt.plot([2048, 2048-6*rho*np.cos(theta)], [2000, 2000-6*rho*np.sin(theta)], lw=2, ls='--', color='magenta')
plt.text(2700, 1700, f"theta = {(theta*180/np.pi):.1f}", color='magenta')
plt.text(x3, y3, "X", size=20, color='red', horizontalalignment='center', verticalalignment='center')
plt.text(x4, y4, "X", size=20, color='red', horizontalalignment='center', verticalalignment='center')
plt.savefig(f"/home/c/cslage/u/Satellites/streak_images/Line_Illustration_{dayObs}_{seqNum}.png")

In [ ]:
theta = np.atan2(x0-x1, y1-y0) * 180.0 / np.pi
theta

In [ ]:
r, c = calexp.image.array.shape

In [ ]:

rho = (x0 - c/2) * np.cos(theta * np.pi / 180.0) + (y0 - r/2) * np.sin(theta * np.pi / 180.0)
rho

In [ ]:
print(r, c)